In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
import quandl

In [2]:
quandl.ApiConfig.api_key = '_66tvRgY5_szTgfB7aeB'
df = quandl.get('BITSTAMP/USD')

In [12]:
df.count()

High          2376
Low           2376
Last          2376
Bid           2376
Ask           2376
Volume        2376
VWAP          2376
Prediction    2346
dtype: int64

In [4]:
predictionDays = 30
# Create another column shifted 'n'  units up
df['Prediction'] = df[['Last']].shift(-predictionDays)
# show the first 5 rows
df.head()

,High,Low,Last,Bid,Ask,Volume,VWAP,Prediction
Date,,,,,,,,
2014-04-15,515.00,453.16,499.01,500.01,505.04,28535.844106,491.41,449.71
2014-04-16,548.00,494.02,534.00,535.01,536.00,31159.941300,520.21,450.67
2014-04-17,537.24,481.63,506.52,504.70,505.38,21126.375080,504.83,446.28
2014-04-18,508.43,470.00,487.00,484.14,487.00,11879.484756,485.72,447.08
2014-04-19,507.43,472.81,504.74,504.74,505.00,10262.195861,492.22,486.98


In [5]:
x = np.array(df.drop(['Prediction'],1))
# Remove the last 'n' rows where 'n' is the predictionDays
x = x[:len(df)-predictionDays]
print(x)

[[  515.           453.16         499.01       ...   505.04
  28535.84410595   491.41      ]
 [  548.           494.02         534.         ...   536.
  31159.9413002    520.21      ]
 [  537.24         481.63         506.52       ...   505.38
  21126.37508023   504.83      ]
 ...
 [11118.49       10826.4        11076.94       ... 11082.98
   5417.56137456 11009.86      ]
 [11496.06       11050.         11356.85       ... 11352.65
   3119.29338641 11339.39      ]
 [11447.44       11267.31       11355.49       ... 11362.12
   2292.87300784 11363.21      ]]


In [6]:
y = np.array(df['Prediction'])
# Get all the values except last 'n' rows
y = y[:-predictionDays]
print(y)

[  449.71   450.67   446.28 ... 15489.15 15332.04 15313.65]


In [7]:
# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size = 0.2)
# set the predictionDays array equal to last 30 rows from the original data set
predictionDays_array = np.array(df.drop(['Prediction'],1))[-predictionDays:]
print(predictionDays_array)

[[11731.7        11175.99       11661.36       11661.46
  11667.07        6732.43099261 11458.17      ]
 [11665.09       11314.53       11439.85       11439.91
  11445.22        5463.38605123 11443.7       ]
 [11555.1        11287.42       11401.81       11401.73
  11406.5         3827.62739785 11408.37      ]
 [11623.6        11250.01       11484.49       11484.13
  11488.91        4804.96936807 11411.54      ]
 [11548.65       11200.         11351.75       11351.86
  11355.32        4381.89502191 11339.34      ]
 [11415.48       11264.01       11343.33       11343.34
  11350.21        1142.49759921 11340.44      ]
 [11500.         11341.55       11468.46       11462.61
  11466.98         836.39097917 11433.35      ]
 [11840.         11409.94       11742.63       11740.8
  11744.1         5634.94342047 11651.92      ]
 [12059.94       11680.         11966.85       11963.68
  11970.78        5759.40054466 11876.87      ]
 [13224.1        11899.1        13224.1        13219.28
  13224.1

In [8]:
from sklearn.svm import SVR
# Create and Train the Support Vector Machine (Regression) using radial basis function
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.00001)
svr_rbf.fit(xtrain, ytrain)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1e-05,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [9]:
svr_rbf_confidence = svr_rbf.score(xtest,ytest)
print('SVR_RBF accuracy :',svr_rbf_confidence)

SVR_RBF accuracy : 0.5502049911712905


In [10]:
# print the predicted values
svm_prediction = svr_rbf.predict(xtest)
print(svm_prediction)
print()
print(ytest)

[5666.24589907 5179.33295038  402.18778269 4940.97974647 4579.21251616
  592.7934068  5417.42545548 6011.51509428 5476.95030012 5194.8122714
 2151.37917511 5002.12862955  739.22128842 1201.37776388 6618.4757752
 6520.23744623 5802.66565917 5175.71847597 6188.36923688 5393.86709248
  569.17606451 6044.71009144 7198.20082898  341.65325581 5190.66971788
 5175.72300693 5180.20884698 5868.00292133 2495.23931354 4077.94092466
 1096.87710717 6597.24978665 3401.7058442  5175.71847597 1049.42546921
  417.24003148  785.27808535  838.96365525  445.17138135 1788.29080864
 5175.83401706  966.42905507 4175.70383914 5175.71847597 3744.98545173
 5175.86787695 3966.30423429 6776.38822903  171.94783263 6237.17091691
 4556.17098134  858.9993237  4993.95798388  875.99056677 3586.16930316
 4093.22054778 8081.64435525 5175.71847683  585.71267475  221.0180229
 5858.07530916  553.66321098 4895.48860706  964.05459271  413.09091678
 5175.80875407 6738.28726522 1297.38731108 1482.5571043   472.01602393
 6103.203

In [11]:
# Print the model predictions for the next 30 days
svm_prediction = svr_rbf.predict(predictionDays_array)
print(svm_prediction)
print()
#Print the actual price for bitcoin for last 30 days
print(df.tail(predictionDays))
#VWAP - volume avg. weighted price

[6039.94269954 5361.16514407 5207.85652961 5332.1356617  5626.3230047
 5175.72030229 5175.71847635 5651.3024973  5398.43696225 5175.71847597
 5175.71847681 5175.71847597 5175.71847597 5175.71847597 5175.71847597
 5175.73466968 5175.71848155 5175.71851118 5175.71847987 5175.71848739
 5175.71847597 5179.58140062 5176.42576406 5175.71847736 5175.71847597
 5175.71847597 5175.71847598 5175.71847597 5175.72097081 5175.71847597]

                High       Low      Last       Bid       Ask        Volume  \
Date                                                                         
2020-10-12  11731.70  11175.99  11661.36  11661.46  11667.07   6732.430993   
2020-10-13  11665.09  11314.53  11439.85  11439.91  11445.22   5463.386051   
2020-10-14  11555.10  11287.42  11401.81  11401.73  11406.50   3827.627398   
2020-10-15  11623.60  11250.01  11484.49  11484.13  11488.91   4804.969368   
2020-10-16  11548.65  11200.00  11351.75  11351.86  11355.32   4381.895022   
2020-10-17  11415.48  11264